In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
df = pd.read_csv('../input/facial-expression/fer2013.csv')
len(df.iloc[0]['pixels'].split())
# 48*48 = 2304
df

In [ ]:
label_map = ['Anger', 'Fear', 'Happy', 'Sad']


In [ ]:
img = df.iloc[0]['pixels'].split()
img = [int(i) for i in img]
type(img[0])
img = np.array(img)
img = img.reshape(48,48)
# print(img.shape)

plt.imshow(img, cmap='gray')
plt.xlabel(df.iloc[0]['emotion'])

In [ ]:
X = []
y = []


In [ ]:
def getData(path):
    anger = 0
    fear = 0
    sad = 0
    happy = 0
    
  
    df = pd.read_csv(path)

    X = []
    y = []

    for i in range(len(df)):
        if df.iloc[i]['emotion'] != 1:
            if df.iloc[i]['emotion'] == 0:
                if anger <= 5000:
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    anger += 1
                else:
                    pass

            if df.iloc[i]['emotion'] == 2:
                if fear <= 5000:
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    fear += 1
                else:
                    pass

            if df.iloc[i]['emotion'] == 3:
                if happy <= 5000:
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    happy += 1
                else:
                    pass

            if df.iloc[i]['emotion'] == 4:
                if sad <= 5000:
                    y.append(df.iloc[i]['emotion'])
                    im = df.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    sad += 1
                else:
                    pass
#             if df.iloc[i]['emotion'] == 6:
#                 if neutral <= 4000:
#                     y.append(df.iloc[i]['emotion'])
#                     im = df.iloc[i]['pixels']
#                     im = [int(x) for x in im.split()]
#                     X.append(im)
#                     neutral += 1
#                 else:
#                     pass

           
    return X, y

In [ ]:
X, y = getData('../input/facial-expression/fer2013.csv')
np.unique(y, return_counts=True)
X = np.array(X)/255.0
y = np.array(y)
X.shape , y.shape

In [ ]:
y_o =[]
for i in y : 
    if i!=2 and i!=3 and i!=4:
        y_o.append(i)
    elif i==2:
        y_o.append(1)
    elif i==3:
            y_o.append(2)
    elif i==4:
            y_o.append(3)

In [ ]:
for i in range(10):
    r = np.random.randint((1), 20000, 1)[0]
    plt.figure()
    plt.imshow(X[r].reshape(48,48), cmap='gray')
    plt.xlabel(label_map[y_o[r]])

In [ ]:
np.unique(y_o,return_counts=True)
for i in range(5):
    r = np.random.randint((1), 20000, 1)[0]
    plt.figure()
    plt.imshow(X[r].reshape(48,48), cmap='gray')
    plt.xlabel(label_map[y_o[r]])

X = X.reshape(len(X), 48, 48, 1)
# no_of_images, height, width, coloar_map
# X.shape

from keras.utils import to_categorical
y_new = to_categorical(y_o, num_classes=None)
len(y_o), y_new.shape
y_o[150], y_new[150]

from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

model = Sequential()


input_shape = (48,48,1)


model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(128, (5, 5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

## (15, 15) --->  30
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')


In [ ]:
model.fit(X, y_new, epochs=500, batch_size=64, shuffle=True, validation_split=0.2)
model.save('model21.h5')

In [ ]:
model.save('../input/facial-expression/model21.h5')